# `Numba` Tutorial

In this tutorial, we will learn how to use numba to speed up python loops.

To intall conda, type `conda install numba`.

In [26]:
import numpy as np
import numba as na
from numba import jit, njit, prange, set_num_threads

### 1. The Numba's JIT decorators, `@jit`.

First, let's consider a nested loop in python.\

Nested loops are very common in any computational physics problems (i.e. the acceleration calculations in the n-body problem).

In [2]:
def native_python(N):
    value = 0
    for _ in range(N):
        for _ in range(N):
            # some physical calculations, such as acceleration. 
            value += np.tanh(123)
    return value

In [5]:
test_size = 3000

In [7]:
%timeit ans = native_python(N=test_size)

916 µs ± 23.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
ans1 = native_python(N=test_size)
print(ans1)

900.0


the above function takes ~6.43 s with `N=3000` (measured by Kuo-Chuan's desktop computer).

In the above example, the calculation is simply adding np.tanh(123) N times. This is equivalent to

In [9]:
ans2 = np.sum(np.tanh(123)*np.ones(test_size*test_size))

In [10]:
print(ans1==ans2)

True


In [11]:
%timeit np.sum(np.tanh(123)*np.ones(test_size**2))

13.9 µs ± 141 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


the same calculation takes only 19.5 ms with `np.sum()` (x330 speedup).

In eariler lecutres, we have learned that we should use `numpy` and `scipy` to avoid using loops in native python.\
However, it is possible that the calculations inside the for loops cannot find counter part calculations in `numpy` and `scipy` (or not straightforward). 

Numba's Just-in-time (JIT) decoraators is one good solution.


In [12]:
@jit(nopython=True)
def numba_jit(N):
    value = 0
    for _ in range(N):
        for _ in range(N):
            value += np.tanh(123)
    return value

In [13]:
ans3 = numba_jit(N=test_size)
print(ans3)
print(ans1==ans3)

900.0
True


In [14]:
%timeit ans = numba_jit(N=test_size)

1.21 µs ± 3.95 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


With `jit`, it takes 6.47 ms now by just adding one line of code!
Note that the performance could be still a bottle neck when `test_size` is big.

In [ ]:
%timeit ans = numba_jit(N=(test_size*10))

10.5 ms ± 188 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The calculation times increased with N^2.

We could actually further improve it with `njit` and `prange`.

In [ ]:
@njit(parallel=True)
def numba_njit_parallel(N):
    value = 0
    for i in prange(N):
        for j in prange(N):
            value += np.tanh(123)
    return value

note that in the above example, we could not use `for _ in prange(N)`, becasue `_` is not recognitzed by numba in parallel computing. 

In [ ]:
ans4 = numba_njit_parallel(N=test_size)
print(ans1==ans4)

True


In [ ]:
set_num_threads(4)

In [ ]:
%timeit ans = numba_njit_parallel(N=(test_size*10))

2.7 ms ± 48.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


It took 161 ms with 4 threads (x4 speedup).

# Exercise

## Exercise 1: Use numba `jit` and `njit` to speedup the Pi calculation. 

Compare your solutions with `numpy`.

In [ ]:
# @jit(nopython=True)
# def numba_jit(N):
#     xx = np.linspace(-1,1,N)
#     circ = lambda x: np.sqrt(1.-x**2)
#     hh = circ(xx)
#     dx = 2. / (N-1)
#     return 2*np.sum(hh)*dx

## Exercise 2: Speedup your N-body solver.

Now, move back to `2_nbody.ipynb`. Let's speed up our `nbody.py` solver with numba.

In [24]:
@jit(nopython=True)
def calculate_acceleration(self, mass, pos):
    acc =(1e13/86400**2)*np.ones((self.particles,3))
    posx = pos[:,0]
    posy = pos[:,1]
    posz = pos[:,2]
    G    = self.G
    npts = self.nparticles
    for i in range(npts):
        for j in range(npts):
            if(j>i):
                x = (posx[i] - posx[j])
                y = (posy[i] - posy[j])
                z = (posz[i] - posz[j])
                rsq = x**2 + y**2 + z**2
                force = -G*mass[i,0]*mass[j,0]/rsq
                theta = np.arctan2(y,x)
            
                fx = force*np.cos(theta)
                fy = force*np.sin(theta)
                fz = 0.0
                acc[i,0] += fx/mass[i,0]
                acc[i,1] += fy/mass[i,0]
                acc[i,2] += fz/mass[i,0]

                acc[j,0] -= fx/mass[j,0]
                acc[j,1] -= fy/mass[j,0]
                acc[j,2] -= fz/mass[j,0]
    return acc


In [25]:
%timeit ans = calculate_acceleration(self, mass, pos)

NameError: name 'self' is not defined